# AutoGen Základná Ukážka

V tejto ukážke kódu použijete AI Framework [AutoGen](https://aka.ms/ai-agents/autogen) na vytvorenie základného agenta.

Cieľom tejto ukážky je ukázať vám kroky, ktoré neskôr použijeme v ďalších ukážkach kódu pri implementácii rôznych agentových vzorov.


## Importujte potrebné balíky Pythonu


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## Vytvorenie klienta

V tomto príklade použijeme [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) na prístup k LLM.

`model` je definovaný ako `gpt-4o-mini`. Skúste zmeniť model na iný dostupný v GitHub Models marketplace, aby ste videli rôzne výsledky.

Ako rýchly test spustíme jednoduchý prompt - `What is the capital of France`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Definovanie agenta

Teraz, keď sme nastavili `client` a overili, že funguje, vytvorme `AssistantAgent`. Každému agentovi môžeme priradiť:  
**name** - Krátky názov, ktorý bude užitočný pri odkazovaní naň v procesoch s viacerými agentmi.  
**model_client** - Klient, ktorého ste vytvorili v predchádzajúcom kroku.  
**tools** - Dostupné nástroje, ktoré môže agent použiť na splnenie úlohy.  
**system_message** - Metaprompt, ktorý definuje úlohu, správanie a tón LLM.  

Môžete zmeniť systémovú správu, aby ste videli, ako LLM reaguje. `Tools` si podrobnejšie vysvetlíme v lekcii č. 4.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## Spustenie agenta

Nasledujúca funkcia spustí agenta. Používame metódu `on_message` na aktualizáciu stavu agenta s novou správou.

V tomto prípade aktualizujeme stav s novou správou od používateľa, ktorá je `"Naplánuj mi skvelú slnečnú dovolenku"`.

Môžete zmeniť obsah správy, aby ste videli, ako LLM reaguje odlišne.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**Upozornenie**:  
Tento dokument bol preložený pomocou služby AI prekladu [Co-op Translator](https://github.com/Azure/co-op-translator). Hoci sa snažíme o presnosť, prosím, berte na vedomie, že automatizované preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho rodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre kritické informácie sa odporúča profesionálny ľudský preklad. Nie sme zodpovední za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
